In [1]:
import pandas as pd
from newspaper import Article
import nltk
import ast
from pandarallel import pandarallel
import requests
import swifter
from tqdm.auto import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
df = pd.read_csv("NepaliTimes.csv", parse_dates=['created_at'])

In [3]:
def keywords(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        return article.keywords
    except Exception as e:
        return 404

In [4]:
df['keywords'] = df.urls.parallel_apply(keywords)

In [5]:
df.to_csv('NepaliTimes_keywords.csv', index=False)

In [27]:
def retry(row):
    if row.keywords==404:
        return keywords(row.urls)
    return row.keywords

In [28]:
df['keywords'] = df.parallel_apply(retry, axis=1)

In [33]:
df=df[df.keywords!=404]

In [34]:
df.to_csv('NepaliTimes_keywords.csv', index=False)

In [35]:
df

,id,created_at,tweet,urls,likes_count,keywords
0,1376906056824016911,2021-03-30 20:21:00+05:45,"Neglected for decades, #Nepal's Far-West is fi...",https://www.nepalitimes.com/banner/the-sun-ris...,26,"[country, rises, west, nepals, nepal, delhi, f..."
1,1376875102919258113,2021-03-30 18:18:00+05:45,How is it possible for women to feel safe and ...,https://www.nepalitimes.com/opinion/educating-...,14,"[harassment, sons, social, sexual, women, move..."
2,1376843645316304902,2021-03-30 16:13:00+05:45,"“Every goal requires teamwork, positive thinki...",https://www.nepalitimes.com/latest/female-foot...,30,"[goals, cup, final, footballers, nepals, win, ..."
3,1376810929963196416,2021-03-30 14:03:00+05:45,"#editorial In a remote part of #Nepal, local e...",https://www.nepalitimes.com/editorial/devoluti...,10,"[rural, devolution, health, later, hospital, n..."
4,1376782744001253378,2021-03-30 12:11:00+05:45,Nepali migrant workers have been waiting for m...,https://www.nepalitimes.com/here-now/delayed-d...,8,"[korea, dreams, oli, nepal, studied, korean, s..."
...,...,...,...,...,...,...
5637,948090231043117056,2018-01-02 13:00:00+05:45,EDITORIAL 2018 should be when a new federal go...,http://archive.nepalitimes.com/article/editori...,12,"[middlemen, nepals, recruiters, export, young,..."
5638,948056690443689986,2018-01-02 10:46:43+05:45,The story of specialty coffee starts at the #f...,http://archive.nepalitimes.com/article/Nepali-...,9,"[coffees, beans, wave, nepal, nepals, qgrader,..."
5639,947788241394130944,2018-01-01 17:00:00+05:45,Happy 2018! #MISSMOTIVATION With the new day c...,http://archive.nepalitimes.com/regular-columns...,8,"[daywith, eleanor, strength, roosevelt, motiva..."
5640,947757617299914752,2018-01-01 14:58:19+05:45,"The air was thick with laughter in Tundikhel, ...",http://archive.nepalitimes.com/blogs/mycity/20...,14,"[lhosar, text, blog, traditional, thick, tamu,..."
